In [55]:
import numpy as np
import pandas as pd


In [56]:
data = pd.read_csv("Dropbox/Kaggle Titanic/train.csv")

In [57]:
data.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [58]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


# learning goals
 

#####  --> how to pre-process data ??
             >dropping not useful feature 
             > filling the missing values(Data Imputation)
#####--> creating a binary decision tree from scratch


In [59]:
##dropping some features


In [60]:
columns_to_drop=["PassengerId","Name","Ticket","Cabin","Embarked"]

In [61]:
data_clean=data.drop(columns_to_drop,axis=1)

In [62]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


### covert string feature to numerical(sex column)

In [63]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_clean["Sex"]=le.fit_transform(data_clean["Sex"])

In [64]:
data_clean.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [65]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


### fill the empty entries(of age column) with mean of the remaining values

In [66]:
data_clean= data_clean.fillna(data_clean["Age"].mean())

In [67]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [68]:
##for accessing 2nd row in pandas

In [69]:
data_clean.loc[2]

Survived     1.000
Pclass       3.000
Sex          0.000
Age         26.000
SibSp        0.000
Parch        0.000
Fare         7.925
Name: 2, dtype: float64

In [70]:
##divide data in input and output column

In [71]:
input_cols=["Pclass","Sex","Age","SibSp","Parch","Fare"]

In [72]:
output_cols=["Survived"]

In [73]:
x=data_clean[input_cols]
y=data_clean[output_cols]


In [74]:
print(x.shape,y.shape)

(891, 6) (891, 1)


In [75]:
print(type(x))

<class 'pandas.core.frame.DataFrame'>


#### Define entropy and information gain

In [76]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    N=float(col.shape[0])
    ent=0.0
    for ix in counts[1]:
        p=ix/N
        ent+=(-1.0*p*np.log2(p))
    return  ent  

In [77]:
### splitting the data about certain feature(fkey) and threshold(fval)

In [78]:
def divide_data(x_data,fkey,fval):
    #work with pandas data frame
    x_right=pd.DataFrame([],columns=x_data.columns)  ##we have made empty list for right node
    x_left=pd.DataFrame([],columns=x_data.columns)   ## for left node
    ##copying the data
    for ix in range (x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
        else:
            x_left=x_left.append(x_data.loc[ix])
    return x_left,x_right        

In [79]:
##x_left,x_right=divide_data(data_clean[:10],'Sex',0.5)
#print(x_left)
#print(x_right)

In [80]:
def information_gain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    # % of total samples are on left and right
    l=float(left.shape[0])/x_data.shape[0]
    r= float(right.shape[0])/x_data.shape[0]
    
    # if all examples come to one side!
    if left.shape[0]==0 or right.shape[0]==0:
        return -1000000 # Minimum information gain
    i_gain=entropy(x_data.Survived)-(1*entropy(left.Survived)+r*entropy(right.Survived))
    return i_gain
    

In [81]:
#test our function
for fx in x.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
-0.4700038680429278
Sex
-0.3157281959851014
Age
-0.553348794259039
SibSp
-0.28576702031361945
Parch
-0.20653035489023408
Fare
-0.17141791178810128


In [82]:
class DecisionTree:
    ##constucter
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
    def     
        
        

SyntaxError: invalid syntax (<ipython-input-82-a4edc9df0ee4>, line 3)